In [ ]:
!pip install dask s3fs xarray zarr

In [ ]:
from dask import distributed
import dask.array as da
import s3fs
import os

In [ ]:
user = os.environ["JUPYTERHUB_USER"]

## Connect to object store and check the content of a folder

In [ ]:
# see https://www.zonca.dev/posts/2022-04-04-zarr_jetstream2 for creating credentials for your own allocation

KEY = os.environ["OS_S3_KEY"]
SECRET = os.environ["OS_S3_SECRET"]

In [ ]:
fs = s3fs.S3FileSystem(
    key=KEY,
    secret=SECRET,
    use_ssl=True,
    client_kwargs=dict(
        endpoint_url="https://js2.jetstream-cloud.org:8001/",
        region_name="RegionOne",
    ),
)

print(fs.ls("gateways23"))

## Create a data store compatible with xarray

In [ ]:
dataset_path = f"gateways23/zarrtest-{user}"

In [ ]:
try:
    fs.rm(dataset_path, recursive=True)
except FileNotFoundError:
    pass

In [ ]:
store = s3fs.S3Map(dataset_path, s3=fs)

## Create an empty Zarr array

In [ ]:
import zarr

z = zarr.empty(
    shape=(1000, 1000), chunks=(100, 100), dtype="f4", store=store, compression=None
)

## Create a random array in Dask

In [ ]:
x = da.random.random(size=z.shape, chunks=z.chunks).astype(z.dtype)

## Store it in Zarr

In [ ]:
x.store(z, lock=False)

## Check the content

In [ ]:
fs.ls(dataset_path)[:5]